<a href="https://colab.research.google.com/github/GadirajuSanjayvarma/Movie-Sentiment-Classification/blob/main/movie_sentiment_generation_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import random
import re
from collections import Counter
from functools import partial
from pathlib import Path


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk import wordpunct_tokenize
from tqdm import tqdm, tqdm_notebook
from sklearn.metrics import classification_report

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


from IPython.core.display import display, HTML
from google_drive_downloader import GoogleDriveDownloader as gdd

# Make plots look pretty
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']
plt.style.use('ggplot')

tqdm.pandas()

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
DATA_PATH = 'data/imdb_reviews.csv'
if not Path(DATA_PATH).is_file():
    gdd.download_file_from_google_drive(
        file_id='1zfM5E6HvKIe7f3rEt1V2gBpw5QOSSKQz',
        dest_path=DATA_PATH,
    )

In [ ]:
def pad_sequences(sequences, padding_val=0, pad_left=False,max_len=241):
    """Pad a list of sequences to the same length with a padding_val."""
    sequence_length = max_len
    if not pad_left:
        result=[]
        for seq in sequences:
          list1=seq+ (sequence_length - len(seq)) * [padding_val]
          result.append(list1[:max_len]) 
        
        return result

In [ ]:
def remove_rare_words(tokens, common_tokens, max_len):
    return [token if token in common_tokens
            else '<UNK>' for token in tokens][-max_len:]


def tokenize(text, stop_words):
    text = re.sub(r'[^\w\s]', '', text)
    #text = re.sub(r'[^\w\s]', '', text)
    text = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", text)
    text = text.lower()
    tokens = wordpunct_tokenize(text)
    tokens = [token for token in tokens if token not in stop_words]
    return tokens


class ImdbSentimentDataset(Dataset):
    def __init__(self, data_path, max_vocab, max_len):
        df = pd.read_csv(data_path)

        # Clean and tokenize
        stop_words = set(stopwords.words('english')) 
        df['tokens'] = df.review.progress_apply(
            partial(tokenize, stop_words=stop_words),
        )
        
        # Replace rare words with <UNK>
        all_tokens = [sublst for lst in df.tokens.tolist() for sublst in lst]

        common_tokens = set(list(zip(
            *Counter(all_tokens).most_common(max_vocab)))[0])
        self.vocab = common_tokens
        self.token2idx = {token: idx for idx, token in enumerate(self.vocab)}
        
        # Add a padding idx
        self.token2idx['<PAD>'] = max(self.token2idx.values()) + 1
        self.token2idx['<EOS>'] = max(self.token2idx.values()) + 1
        
        self.idx2token = {idx: token for token, idx in self.token2idx.items()}
        
        df['indexed_tokens'] = df.tokens.apply(
            lambda tokens: [self.token2idx[token] for token in tokens if token in self.token2idx],
        )
        self.text = df.review.tolist()
 
        self.sequences=pad_sequences(sequences=df.indexed_tokens.tolist(),padding_val=self.token2idx['<PAD>'],max_len=max_len)
        
        target_virtual=[self.sequences[i][1:] for i in range(0,len(self.sequences))]
        for t in range(0,len(target_virtual)):
          target_virtual[t].append(self.token2idx['<EOS>'])
        self.targets =target_virtual 

    def __getitem__(self, i):
        #print(self.sequences[i])
        #print(self.targets[i])
        #print(self.text[i])
        return self.sequences[i], self.targets[i],  self.text[i]
    
    def __len__(self):
        return len(self.sequences)

In [ ]:
dataset.idx2token[1000]

'19172000br'

In [ ]:
print(len(dataset.sequences[25]))
print(len(dataset.targets[65]))

35
35


In [ ]:
max_vocab=100
max_len=5

In [ ]:
dataset = ImdbSentimentDataset(DATA_PATH, max_vocab, max_len)

100%|██████████| 62155/62155 [00:12<00:00, 4868.58it/s]


In [ ]:
def split_train_valid_test(corpus, valid_ratio=0.1, test_ratio=0.1):
    """Split dataset into train, validation, and test."""
    test_length = int(len(corpus) * test_ratio)
    valid_length = int(len(corpus) * valid_ratio)
    train_length = len(corpus) - valid_length - test_length
    return random_split(
        corpus, lengths=[train_length, valid_length, test_length],
    )

In [ ]:
valid_ratio = 0.05  #@param {type:"slider", min:0.01, max:0.3, step:0.01}
test_ratio = 0.05  #@param {type:"slider", min:0.01, max:0.3, step:0.01}

train_dataset, valid_dataset, test_dataset = split_train_valid_test(
    dataset, valid_ratio=valid_ratio, test_ratio=test_ratio)
len(train_dataset), len(valid_dataset), len(test_dataset)

(55941, 3107, 3107)

In [ ]:
def collate(batch):
    inputs = [item[0] for item in batch]
    
    target =torch.LongTensor([item[1] for item in batch])
    text = [item[2] for item in batch]
    return inputs, target, text

# Powers of two are preferred for optimal usage on the GPU
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, collate_fn=collate)
test_loader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate)

In [ ]:
dataset.idx2token[949]

'second'

In [ ]:
for index,seq in enumerate(dataset.sequences):
  if(len(seq)!=241):
    print("hi")
    print(index)
    break

In [ ]:
for inputs,target,text in train_loader:
  print(inputs[15])
  print(target[15])
  print(text[15])
  break

[91, 45, 75, 5, 48]
tensor([ 45,  75,   5,  48, 101])
'Capital City' fans rejoice! This first season of this series is now available from Network DVD and I've recently got my copy! Although very much an ensemble piece of key 'maverick' trading floor characters 'CAPITAL CITY' does present us with various moments through both its first and second season when each member of the team plays a significant part in a particular central or peripheral plot line. The cultural mix (English, Irish, American, German, Polish) of Head Trader Wendy Foley's (played by Joanna Phillips-Lane) group of staff is balanced with their own distinctive mannerisms, interests and personalities which helps to make the rather unfamiliar and, to most people, seemingly sterile subject of financial trading reasonably engaging through the engaging performances of the cast. In fact this seemingly dynamic young team of employees is in direct contrast to the rather staid and old-fashioned senior management of Shane Longman 

In [ ]:
def pad_sequences(sequences, padding_val=0, pad_left=False):
    """Pad a list of sequences to the same length with a padding_val."""
    sequence_length = max(len(sequence) for sequence in sequences)
    if not pad_left:
        return [
            sequence + (sequence_length - len(sequence)) * [padding_val]
            for sequence in sequences
        ]
    return [
        (sequence_length - len(sequence)) * [padding_val] + sequence
        for sequence in sequences
    ]


class RNNClassifier(nn.Module):
    def __init__(self, output_size, hidden_size, vocab_size, padding_idx,
                 device, dropout_probability=0.3, bidirectional=False, n_layers=1,
                 embedding_dimension=50, batch_size=32):
        super(RNNClassifier, self).__init__()
        self.output_size = output_size
        self.batch_size = batch_size
        
        self.dropout_probability = dropout_probability
        self.device = device
        self.padding_idx = padding_idx
        
        # We need to multiply some layers by two if the model is bidirectional
        self.input_size_factor = 2 if bidirectional else 1
        self.hidden_size = hidden_size//self.input_size_factor
        self.n_layers = n_layers
        self.embedding = nn.Embedding(vocab_size, embedding_dimension)
        
        self.rnn = nn.LSTM(
            embedding_dimension,
            self.hidden_size,
            self.n_layers,
            bidirectional=bidirectional,
        )

        
    def init_hidden(self):
        """Set initial hidden states."""
        h0 = torch.randn(
            self.n_layers* self.input_size_factor,
            self.batch_size,
            self.hidden_size,
        )
        c0 = torch.randn(
            self.n_layers* self.input_size_factor,
            self.batch_size,
            self.hidden_size,
        )
        
        h0 = h0.to(self.device)
        c0 = c0.to(self.device)

        return h0, c0
    
    def apply_rnn(self, embedding_out):
        
        activations, (hn,cn) = self.rnn(embedding_out, self.init_hidden())
        
        return activations

    def forward(self, inputs, return_activations=False):
        batch_size = len(inputs)
    
        # This makes the model not break for the last batch that might be less
        # than batch_size in size
        if batch_size != self.batch_size:
            self.batch_size = batch_size

        inputs = torch.LongTensor(inputs).to("cuda")
        inputs=inputs.transpose(0,1)
        
        # Get embeddings
        embedding_out = self.embedding(inputs)
        
        activations = self.apply_rnn(embedding_out)
        
        out = torch.sigmoid(activations)

        # Put the output back in correct order
        return out

In [ ]:
def train_epoch(model, optimizer, scheduler, train_loader):
    model.train()
    total_loss = total = 0
    
    progress_bar = tqdm_notebook(train_loader, desc='Training', leave=False)
    for inputs, target, text in progress_bar:
        #target=torch.tensor(target)
        target = target.to(device)
        #target=target.transpose(0,1)
        # Clean old gradients
        optimizer.zero_grad()
        
        # Forwards pass
        output = model(inputs)
        outputs=output.transpose(0,1)
        #print(outputs)
        #print(outputs.shape)
        #print()
        outputs=outputs.permute(0,2,1)
        #output=output.transpose(0,1).permute(0,2,1).clone()
        
        loss = nn.CrossEntropyLoss()(outputs, target)

        # Perform gradient descent, backwards pass
        loss.backward()

        # Take a step in the right direction
        optimizer.step()
        scheduler.step()

        # Record metrics
        total_loss += loss.item()
        total += len(target)
        progress_bar.set_description(
        f'train_loss: {loss:.2e}'
        f'\tavg_loss: {total_loss/total:.2e}\n',
      )

    return total_loss / total


def validate_epoch(model, valid_loader):
    model.eval()
    total_loss = total = 0
    with torch.no_grad():
        progress_bar = tqdm_notebook(valid_loader, desc='Validating', leave=False)
        for inputs, target, text in progress_bar:
            target = target.to(device)

            # Forwards pass
            output = model(inputs)
            outputs=output.transpose(0,1)
            #print(outputs)
            #print(outputs.shape)
            #print()
            outputs=outputs.permute(0,2,1)
            #output=output.transpose(0,1).permute(0,2,1).clone()
            
            loss = nn.CrossEntropyLoss()(outputs, target)

            # Record metrics
            total_loss += loss.item()
            total += len(target)

    return total_loss / total

In [ ]:
print(len(dataset.idx2token))

1002


In [ ]:
dropout_probability = 0.2  #@param {type:"slider", min:0.0, max:0.8, step:0.1}
n_rnn_layers = 1  #@param {type:"integer"}
embedding_dimension = 20  #@param {type:"integer"}
hidden_size = len(dataset.token2idx)  #@param {type:"integer"}
is_bidirectional = False  #@param ["False", "True"] {type:"raw"}
max_epochs = 10  #@param {type:"integer"}
learning_rate = 0.001  #@param {type:"number"}
input_size_factor = 2 if is_bidirectional else 1
model = RNNClassifier(
    output_size=len(dataset.token2idx),  # two classes: positive vs. negative
    hidden_size=hidden_size,
    embedding_dimension=embedding_dimension,
    vocab_size=len(dataset.token2idx),
    padding_idx=dataset.token2idx['<PAD>'],
    dropout_probability=dropout_probability,
    bidirectional=is_bidirectional,
    n_layers=n_rnn_layers,
    device=device,
    batch_size=batch_size,
)
model = model.to(device)
print(model)
pytorch_total_params = sum(p.numel() for p in model.parameters())
print("total parameters are",pytorch_total_params)

RNNClassifier(
  (embedding): Embedding(102, 20)
  (rnn): LSTM(20, 102)
)
total parameters are 52632


In [ ]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
print("total parameters are",pytorch_total_params)

total parameters are 9202368


In [ ]:
for p in model.parameters():
  print(p)
  break

Parameter containing:
tensor([[ 1.4529,  0.5179,  0.3490,  ...,  1.0360,  0.3178,  0.2285],
        [ 0.8345,  1.4144, -1.2868,  ..., -0.0245,  1.6150,  0.2457],
        [-1.8814, -1.2977,  0.2854,  ...,  0.2528,  0.2496,  1.3550],
        ...,
        [ 0.9046,  0.2742, -1.6734,  ..., -0.5467, -1.9016, -0.8535],
        [ 0.7058, -0.2564,  1.0042,  ...,  0.6544,  0.5006,  1.4197],
        [ 0.1687, -1.6029,  0.0117,  ...,  0.3203, -0.4886,  0.0849]],
       device='cuda:0', requires_grad=True)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=0.1,
)
scheduler = CosineAnnealingLR(optimizer, 1)

In [ ]:
def get_hidden():
        """Set initial hidden states."""
        h0 = torch.zeros(
            n_rnn_layers* input_size_factor,
            1,
            hidden_size//input_size_factor,
        )
        c0 = torch.zeros(
            n_rnn_layers* input_size_factor,
            1,
            hidden_size//input_size_factor,
        )
        
        h0 = h0.to(device)
        c0 = c0.to(device)

        return h0, c0

In [ ]:
def print_re(text,hiddens):
  
  print(text,end=' ')
  tokens=parse_string(text,dataset.token2idx)
  #print(tokens)
  #(h,c)=get_hidden()
  model.eval()
  with torch.no_grad():
        # Forwards pass
        
        for i in range(0,5):
          #print(tokens)
          #print(h)
          embeddings=model.embedding(tokens)
          hidden_output,(h,c)=model.rnn(embeddings,hiddens)
          #print(hidden_output.shape)
          hidden_output = torch.sigmoid(hidden_output)
          hidden_output=hidden_output[0].squeeze(0).squeeze(0)
          #print(hidden_output.shape)
          prediction = hidden_output.cpu().numpy()
          #print(logits.shape)
          prediction = np.argmax(prediction, axis=0)
          #print("prediction is ",prediction)
          print(dataset.idx2token[prediction],end=' ')
          #print(hidden_output)
          tokens=torch.LongTensor([[prediction]]).to("cuda")

In [ ]:
(h0,c0)=get_hidden()

In [ ]:
print_re("one",(h0,c0))

one movies <EOS> <EOS> <EOS> <EOS> 

In [ ]:
print(dataset.vocab)

{'dont', 'get', 'best', 'say', 'part', 'much', 'know', 'want', 'great', 'made', 'two', 'bad', 'scenes', 'cant', 'love', 'better', 'scene', 'another', 'like', 'character', 'go', 'see', 'cast', 'fact', 'acting', 'years', 'show', 'however', 'way', 'director', 'around', 'could', 'something', 'actors', 'find', 'many', 'though', 'actually', 'ever', 'films', 'young', 'things', 'big', 'one', 'think', 'ive', 'makes', 'funny', 'characters', 'doesnt', 'im', 'lot', 'us', 'end', 'well', 'br', 'people', 'story', 'give', 'really', 'life', 'may', 'every', 'would', 'film', 'still', 'make', 'movie', 'enough', 'thats', 'man', 'thing', 'world', 'seems', 'little', 'got', 'time', 'plot', 'watching', 'pretty', 'take', 'new', 'didnt', 'never', 'back', 'also', 'work', 'watch', 'real', 'even', 'good', 'first', 'seen', 'quite', 'movies', 'old', 'horror', 'going', 'nothing', 'look'}


In [ ]:
train_losses, valid_losses = [], []
n_epochs=0
for _ in range(20):
    print()
    print()
    print_re("character",get_hidden())
    print()
    print()
    print()
    train_loss = train_epoch(model, optimizer, scheduler, train_loader)
    valid_loss = validate_epoch(model, valid_loader)
    #print("model parameters")
    print()
    print()
    print()
    for p in model.parameters():
      print(p)
      print(p.grad)
      break
    print()
    print()
    print()
    tqdm.write(
        f'epoch #{n_epochs + 1:3d}\ttrain_loss: {train_loss:.2e}'
        f'\tvalid_loss: {valid_loss:.2e}\n',
    )
    
    # Early stopping if the current valid_loss is greater than the last three valid losses
    train_losses.append(train_loss)
    #valid_losses.append(valid_loss)
    n_epochs+=1
   



character something ever films great say 




/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`





Parameter containing:
tensor([[-2.1657, -1.7539,  0.6470,  ...,  0.1411,  0.0239, -0.2495],
        [-1.8883, -1.6276, -0.6661,  ..., -0.5254, -0.4992, -2.5066],
        [ 0.0972, -0.6541, -0.1728,  ...,  1.5574, -1.4388,  0.2042],
        ...,
        [-1.1703, -1.1040, -1.9903,  ...,  0.9157,  0.3449, -1.1825],
        [ 0.4763,  0.6482,  0.3027,  ..., -1.0570,  0.8429, -1.6645],
        [-0.2114,  1.3066, -0.6469,  ..., -0.3320,  1.5027, -1.7053]],
       device='cuda:0', requires_grad=True)
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')



epoch #  1	train_loss: 7.20e-02	valid_loss: 7.24e-02



character <EOS> back <EOS> back <EOS> 







Parameter containing:
tensor([[-2.1666, -1.7540,  0.6473,  ...,  0.1416,  0.0234, -0.2501],
        [-1.8889, -1.6276, -0.6658,  ..., -0.5253, -0.4996, -2.5069],
        [ 0.0964, -0.6544, -0.1723,  ...,  1.5579, -1.4398,  0.2039],
        ...,
        [-1.1709, -1.1041, -1.9901,  ...,  0.9157,  0.3446, -1.1826],
        [ 0.4752,  0.6480,  0.3032,  ..., -1.0564,  0.8424, -1.6649],
        [-0.2114,  1.3066, -0.6469,  ..., -0.3320,  1.5027, -1.7053]],
       device='cuda:0', requires_grad=True)
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')



epoch #  2	train_loss: 7.17e-02	valid_loss: 7.22e-02



character <EOS> back <EOS> back <EOS> 




...




Parameter containing:
tensor([[-2.1813, -1.7575,  0.6526,  ...,  0.1500,  0.0146, -0.2590],
        [-1.8982, -1.6282, -0.6616,  ..., -0.5220, -0.5048, -2.5094],
        [ 0.0847, -0.6590, -0.1626,  ...,  1.5700, -1.4620,  0.1954],
        ...,
        [-1.1775, -1.1059, -1.9872,  ...,  0.9168,  0.3404, -1.1839],
        [ 0.4721,  0.6480,  0.3070,  ..., -1.0536,  0.8451, -1.6685],
        [-0.2114,  1.3066, -0.6469,  ..., -0.3320,  1.5027, -1.7053]],
       device='cuda:0', requires_grad=True)
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')



epoch # 16	train_loss: 7.06e-02	valid_loss: 7.11e-02



character <EOS> movie <EOS> movie <EOS> 







Parameter containing:
tensor([[-2.1825, -1.7578,  0.6530,  ...,  0.1506,  0.0138, -0.2597],
        [-1.8989, -1.6283, -0.6613,  ..., -0.5218, -0.5051, -2.5095],
        [ 0.0840, -0.6594, -0.1618,  ...,  1.5713, -1.4644,  0.1947],
        ...,
        [-1.1780, -1.1060, -1.9870,  ...,  0.9169,  0.3401, -1.1840],
        [ 0.4722,  0.6481,  0.3073,  ..., -1.0536,  0.8458, -1.6689],
        [-0.2114,  1.3066, -0.6469,  ..., -0.3320,  1.5027, -1.7053]],
       device='cuda:0', requires_grad=True)
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')



epoch # 17	train_loss: 7.05e-02	valid_loss: 7.11e-02



character <EOS> movie <EOS> movie <EOS> 







Parameter containing:
tensor([[-2.1836, -1.7582,  0.6535,  ...,  0.1512,  0.0130, -0.2602],
        [-1.8995, -1.6283, -0.6610,  ..., -0.5216, -0.5056, -2.5097],
        [ 0.0832, -0.6599, -0.1609,  ...,  1.5726, -1.4667,  0.1941],
        ...,
        [-1.1785, -1.1062, -1.9868,  ...,  0.9170,  0.3397, -1.1841],
        [ 0.4723,  0.6482,  0.3076,  ..., -1.0537,  0.8463, -1.6693],
        [-0.2114,  1.3066, -0.6469,  ..., -0.3320,  1.5027, -1.7053]],
       device='cuda:0', requires_grad=True)
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')



epoch # 18	train_loss: 7.05e-02	valid_loss: 7.10e-02



character <EOS> movie <EOS> movie <EOS> 







Parameter containing:
tensor([[-2.1847, -1.7586,  0.6539,  ...,  0.1519,  0.0123, -0.2608],
        [-1.9002, -1.6284, -0.6607,  ..., -0.5214, -0.5059, -2.5098],
        [ 0.0826, -0.6604, -0.1602,  ...,  1.5740, -1.4692,  0.1936],
        ...,
        [-1.1790, -1.1063, -1.9865,  ...,  0.9171,  0.3393, -1.1842],
        [ 0.4725,  0.6483,  0.3078,  ..., -1.0538,  0.8469, -1.6697],
        [-0.2114,  1.3066, -0.6469,  ..., -0.3320,  1.5027, -1.7053]],
       device='cuda:0', requires_grad=True)
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')



epoch # 19	train_loss: 7.04e-02	valid_loss: 7.10e-02



character <EOS> movie <EOS> movie <EOS> 







Parameter containing:
tensor([[-2.1858, -1.7590,  0.6543,  ...,  0.1525,  0.0115, -0.2614],
        [-1.9007, -1.6284, -0.6603,  ..., -0.5213, -0.5062, -2.5100],
        [ 0.0820, -0.6608, -0.1593,  ...,  1.5754, -1.4716,  0.1930],
        ...,
        [-1.1795, -1.1065, -1.9864,  ...,  0.9172,  0.3390, -1.1843],
        [ 0.4726,  0.6484,  0.3080,  ..., -1.0539,  0.8474, -1.6701],
        [-0.2114,  1.3066, -0.6469,  ..., -0.3320,  1.5027, -1.7053]],
       device='cuda:0', requires_grad=True)
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')



epoch # 20	train_loss: 7.04e-02	valid_loss: 7.09e-02



In [ ]:
for p in model.parameters():
  print(p.grad)
  break

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')


### model testing for generation

In [ ]:
for inputs,targets,text in train_loader:
  #print(torch.LongTensor(inputs).shape)
  #print(inputs)
  #print()
  outputs=model(inputs)
  #print(outputs.shape)
  #print(outputs)
  #print()
  outputs=outputs.transpose(0,1)
  #print(outputs)
  #print(outputs.shape)
  #print()
  outputs=outputs.permute(0,2,1)
  #print(outputs)
  #print(outputs.shape)
  #print()

  #print(targets)
  #print(targets.shape)
  print(nn.CrossEntropyLoss()(outputs,targets.to("cuda")))

  break


tensor(6.9082, device='cuda:0', grad_fn=<NllLoss2DBackward>)


In [ ]:
def parse_string(text,dictionary):

  stop_words = set(stopwords.words('english')) 
  tokens=tokenize(text,stop_words=stop_words)

  result=[]
  for token in tokens:
     if(token in dictionary):
       result.append(dictionary[token])
     else:
       result.append(dictionary['<UNK>'])
       

  return torch.LongTensor([[result[-1]]]).to("cuda")


In [ ]:
print(type(dataset.vocab))

<class 'list'>


In [ ]:
 def get_hidden():
        """Set initial hidden states."""
        h0 = torch.randn(
            1,
            1,
            len(dataset.token2idx),
        )
        c0 = torch.randn(
            1,
            1,
            len(dataset.token2idx),
        )
        
        h0 = h0.to(device)
        c0 = c0.to(device)

        return h0, c0

In [ ]:
a=torch.rand(2,5,3)
print(a)
a=a.transpose(0,1)
print(a)
a=a.transpose(0,1)
print(a.shape)
a=a.transpose(0,2,1)
print(a)


tensor([[[0.9132, 0.7301, 0.4016],
         [0.8519, 0.0190, 0.0787],
         [0.9244, 0.4025, 0.9474],
         [0.5818, 0.6863, 0.2204],
         [0.0573, 0.9797, 0.7357]],

        [[0.3564, 0.4137, 0.1810],
         [0.4938, 0.8413, 0.2513],
         [0.1513, 0.6442, 0.7123],
         [0.5432, 0.6413, 0.2469],
         [0.1061, 0.4487, 0.2965]]])
tensor([[[0.9132, 0.7301, 0.4016],
         [0.3564, 0.4137, 0.1810]],

        [[0.8519, 0.0190, 0.0787],
         [0.4938, 0.8413, 0.2513]],

        [[0.9244, 0.4025, 0.9474],
         [0.1513, 0.6442, 0.7123]],

        [[0.5818, 0.6863, 0.2204],
         [0.5432, 0.6413, 0.2469]],

        [[0.0573, 0.9797, 0.7357],
         [0.1061, 0.4487, 0.2965]]])
torch.Size([2, 5, 3])


TypeError: ignored

In [ ]:
model.eval()
test_accuracy = n_examples = 0
y_true, y_pred = [], []
with torch.no_grad():
    for inputs, target, text in tqdm_notebook(test_loader, leave=False):
        target = target.to(device)

        logits = model(inputs)

        logits = logits.squeeze(0).detach().cpu().numpy()
        #print(logits.shape)
        predictions = np.argmax(logits, axis=1)
        target = target.cpu().numpy()
        
        y_true.extend(predictions)
        y_pred.extend(target)
        
print(classification_report(y_true, y_pred))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


              precision    recall  f1-score   support

           0       0.85      0.86      0.86      1576
           1       0.85      0.85      0.85      1531

    accuracy                           0.85      3107
   macro avg       0.85      0.85      0.85      3107
weighted avg       0.85      0.85      0.85      3107



In [ ]:
flatten = lambda x: [sublst for lst in x for sublst in lst]
inputs_lst, target_lst, text_lst = zip(*test_loader)
inputs_lst, target_lst, text_lst = map(flatten, [inputs_lst, target_lst, text_lst])
test_examples = list(zip(inputs_lst, target_lst, text_lst))

def print_random_prediction(n=10):
    to_emoji = lambda x: '😄' if x else '😡'
    model.eval()
    rows = []
    for _ in range(n):
        with torch.no_grad():
            inputs, target, text = random.choice(test_examples)
            target = target.item()
            
            logits = model([inputs])
            logits = logits.squeeze(0).detach().cpu().numpy()
            prediction = np.argmax(logits, axis=1)[0]

            predicted = to_emoji(prediction)
            actual = to_emoji(target)
            
            row = f"""
            <tr>
            <td>{text}&nbsp;</td>
            <td>{predicted}&nbsp;</td>
            <td>{actual}&nbsp;</td>
            </tr>
            """
            rows.append(row)
            
    rows_joined = '\n'.join(rows)
    table = f"""
<table>
<tbody>
<tr>
<td><b>Review</b>&nbsp;</td>
<td><b>Predicted</b>&nbsp;</td>
<td><b>Actual</b>&nbsp;</td>
</tr>
{rows_joined}
</tbody>
</table>
"""
    display(HTML(table))

In [ ]:
print_random_prediction(n=10)

In [ ]:
def parse_string(text,dictionary):

  stop_words = set(stopwords.words('english')) 
  tokens=tokenize(text,stop_words=stop_words)

  result=[]
  for token in tokens:
     if(token in dictionary):
       result.append(dictionary[token])
     else:
       result.append(dictionary['<UNK>'])
       

  return [result]


In [ ]:
print(dataset.token2idx['<UNK>'])

10


In [ ]:
def get_prediction(tokens):
  with torch.no_grad():
      logits = model(tokens)
      logits = logits.squeeze(0).detach().cpu().numpy()
      prediction = np.argmax(logits, axis=1)[0]
      if(prediction==1):
        print("positive")
      else:
        print("negative")  


In [ ]:
review='The biggest disaster of super stars career. What does even Srikanth think? If he keeps many great actors on the screen and dont focus on the story, we will see it? Single star goes to the songs and background score, the ⅕th part of a film. Story, screenplay and other vital points are stuffed inside the ground. I hope some improvement. Please dont disappoint like this.'
review_tokens=parse_string(review,dataset.token2idx)
#print(review_tokens)
get_prediction(review_tokens)

negative


In [ ]:
print(review_tokens)

[569, 375, 88]


## saving the model

In [ ]:
torch.save(model.state_dict(), "/content/movie_sentiment.pt")